In [1]:
%pip install tensorflow
%pip install pandas 
%pip install numpy


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import datetime
import os
from feature_engineering.tensor_features import develop_features, floating_conv

from data_handler import LocalToLargeDataLoader
from path_finder import path_sorter

In [4]:
data_loader = LocalToLargeDataLoader(print_progress=True)
parsed_data = data_loader.load_raw_data()


Retrieving training data...


In [5]:
path_dict = path_sorter(parsed_data)

[===================-] 100.0% complete


In [6]:
print(list(path_dict.keys())[0:5])
print(len(list(path_dict.keys())))
print(path_dict[list(path_dict.keys())[0]][0])

[(('61e9f38eb937134a3c4bfd8b',), Timestamp('2024-05-05 13:25:50')), (('61e9f38eb937134a3c4bfd8b',), Timestamp('2024-05-04 22:42:36')), (('61e9f38eb937134a3c4bfd8b',), Timestamp('2024-05-04 16:15:36')), (('61e9f38eb937134a3c4bfd8b',), Timestamp('2024-05-04 05:57:22')), (('61e9f38eb937134a3c4bfd8b',), Timestamp('2024-05-04 02:38:47'))]
152532
                       time  latitude  longitude
1492322 2024-05-05 13:25:50  36.84686    5.84638
1491674 2024-05-05 12:13:37  36.84628    5.84575
1491516 2024-05-05 11:55:38  36.84670    5.84588
1491354 2024-05-05 11:32:14  36.84647    5.84534
1491022 2024-05-05 10:43:25  36.84636    5.84488
1490878 2024-05-05 10:29:25  36.84631    5.84503
1488903 2024-05-05 06:11:48  37.00551    5.81394
1486780 2024-05-05 01:41:11  36.99668    5.79674
1486640 2024-05-05 01:31:47  36.99575    5.79647
1485512 2024-05-04 23:16:12  36.98595    5.79692


In [ ]:
features, y_raw, test_features, y_raw_test = data_loader.load_training_data(path_dict) 

In [8]:
print("Develop features")
feature_array, y = develop_features(features, y_raw)
print("Develop test features")
test_feature_array, y_test = develop_features(test_features, y_raw_test)

Develop features


TypeError: list indices must be integers or slices, not list

In [1]:
print(list(features.keys())[0:5])
print(len(list(features.keys())))
print(features[list(features.keys())[0]][1])
print(feature_array[0])

NameError: name 'features' is not defined

In [12]:

x_train = tf.convert_to_tensor(feature_array)
x_test = tf.convert_to_tensor(test_feature_array)

In [13]:
y_train = np.array(y)
y_test = np.array(y_test)

input_shape = x_test.shape
print(input_shape)

(30507, 34)


In [1]:
parameters=(input_shape[0])
print(parameters)
layer_n=2
layer=[
        tf.keras.layers.Normalization(),
        tf.keras.layers.Dense(parameters//layer_n+1, activation='relu'),
        tf.keras.layers.Dropout(0.2)
        ]

model_list=[tf.keras.Input(shape=input_shape),
            tf.keras.layers.Dense(input_shape[0], activation='relu')]
model_list.extend(layer*layer_n)
model_list.append(tf.keras.layers.Dense(2, activation='linear'))
print(model_list)
model = tf.keras.models.Sequential(model_list)

model.compile(optimizer='adam',
  loss='mean_squared_error',
  metrics=['mae','mape'])

model.summary()

early_stopping = EarlyStopping(
    monitor='val_loss',  # You can change this to 'val_accuracy' or another metric
    patience=5,          # Number of epochs to wait for improvement
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

checkpoint = ModelCheckpoint(
    filepath='modelepoch{epoch:02d}val_loss{val_loss:.2f}.keras',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    verbose=1
)

log_dir= os.path.join('logs','fit',datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),)
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


NameError: name 'input_shape' is not defined

In [26]:
print(f"Y data samples {y_train[:5]}")
model.fit(x_train, y_train, epochs=200, batch_size=32, validation_split=0.2, callbacks=[checkpoint, early_stopping])

Y data samples [[  36.84686    5.84638]
 [  48.5332    -6.12003]
 [  28.82071 -120.46513]
 [  33.86918 -118.81433]
 [  33.18068   -9.43055]]
Epoch 1/20
 8/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 28622121336832.0000 - mae: 4240309.5000 - mape: 38419712.0000  
Epoch 1: val_loss did not improve from 23641094144.00000
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 25887286231040.0000 - mae: 4034683.5000 - mape: 36272012.0000 - val_loss: 775497777152.0000 - val_mae: 811846.8125 - val_mape: 10468310.0000
Epoch 2/20
 1/14 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 22770285019136.0000 - mae: 4008617.0000 - mape: 17482132.0000
Epoch 2: val_loss did not improve from 23641094144.00000
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 18431774556160.0000 - mae: 3425951.7500 - mape: 25047640.0000 - val_loss: 110672863232.0000 - val_mae: 282489.5000 - val_mape: 3566033.5000
Epoch 3/20
13/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 10771739181056.0000 - mae: 2604610.0000 - mape: 21291736.0000  
Epo